# Define CNN Architecture

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        # =========== OLD APPROACH FOR FIXED NUM OF DIFFERENT FILTERS =========== #
        # self.conv_0 = nn.Conv2d(in_channels = 1, 
        #                         out_channels = n_filters, 
        #                         kernel_size = (filter_sizes[0], embedding_dim))
        
        # self.conv_1 = nn.Conv2d(in_channels = 1, 
        #                         out_channels = n_filters, 
        #                         kernel_size = (filter_sizes[1], embedding_dim))
        
        # self.conv_2 = nn.Conv2d(in_channels = 1, 
        #                         out_channels = n_filters, 
        #                         kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        # =========== OLD APPROACH FOR FIXED NUM OF DIFFERENT FILTERS =========== #
        # #embedded = [batch size, 1, sent len, emb dim]
        
        # conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        # conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        # conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        # #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        # pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        # pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        # pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        # cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        return self.fc(cat)

In [8]:
# =========== EXPERIMENT WITH 1D CNN =========== #
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

In [131]:
# import json

# with open('/content/Sarcasm_Headlines_Dataset_v2.json') as data_file:
#     data = json.load(data_file)

# for element in data:
#     if 'article_link' in element:
#         del element['article_link']

# with open('Sarcasm_Headlines_Dataset_v2.json', 'w') as data_file:
#     data = json.dump(data, data_file)

# Sarcasm Data (Headlines)

In [3]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# create fields in emotion dataset
TEXT = data.Field(tokenize = 'spacy', 
                  tokenizer_language = 'en_core_web_sm',
                  batch_first = True
                  )
LABEL = data.LabelField(dtype=torch.float)

# tuples representative of tabular format
fields = [
          ('is_sarcastic', LABEL),
          ('headline', TEXT)
]

# LOAD EMOTION DATA
sarc_train, = data.TabularDataset.splits(
    path='/content',
    train='Sarcasm_Headlines_Dataset_v2.csv',
    format='csv',
    fields=fields,
    skip_header=False
)

In [4]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(sarc_train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(sarc_train)


sarc_train, sarc_valid, sarc_test = sarc_train.split(random_state = random.seed(SEED), split_ratio=[0.4,0.3,0.3])

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 399152/400000 [00:15<00:00, 26579.56it/s]

In [5]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sarc_train_iterator, sarc_valid_iterator, sarc_test_iterator = data.BucketIterator.splits(
    (sarc_train, sarc_valid, sarc_test), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)

In [9]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,620,801 trainable parameters


In [11]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# zero initial weights of <unk> and <pad>
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## TRAINING

In [12]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

###  Helpers

In [13]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [14]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        # print(batch.__dict__)
        optimizer.zero_grad()
        predictions = model(batch.headline).squeeze(1)
        loss = criterion(predictions, batch.is_sarcastic)
        acc = binary_accuracy(predictions, batch.is_sarcastic)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [15]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.headline).squeeze(1)
            loss = criterion(predictions, batch.is_sarcastic)
            acc = binary_accuracy(predictions, batch.is_sarcastic)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Training - Sentiment

In [17]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, sarc_train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, sarc_valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'CNN-sarcasm-baseline.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 0.543 | Train Acc: 72.27%
	 Val. Loss: 0.401 |  Val. Acc: 82.70%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.338 | Train Acc: 85.77%
	 Val. Loss: 0.333 |  Val. Acc: 85.33%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.209 | Train Acc: 92.23%
	 Val. Loss: 0.327 |  Val. Acc: 85.64%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.126 | Train Acc: 95.76%
	 Val. Loss: 0.350 |  Val. Acc: 85.74%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.070 | Train Acc: 98.03%
	 Val. Loss: 0.367 |  Val. Acc: 86.16%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 0.040 | Train Acc: 99.04%
	 Val. Loss: 0.414 |  Val. Acc: 85.82%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 0.022 | Train Acc: 99.58%
	 Val. Loss: 0.445 |  Val. Acc: 85.81%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 0.013 | Train Acc: 99.79%
	 Val. Loss: 0.483 |  Val. Acc: 85.54%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 0.009 | Train Acc: 99.89%
	 Val. Loss: 0.527 |  Val. Acc: 85.19%
Epoch: 10 | Epoch Time: 0m 1

In [18]:
# Validation
model.load_state_dict(torch.load('CNN-sarcasm-baseline.pt'))

test_loss, test_acc = evaluate(model, sarc_test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.326 | Test Acc: 85.87%


## Test - Sentiment

In [21]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sarcasm(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [41]:
predict_sarcasm(model, "")

0.49969905614852905